In [1]:
import torch 
import torch.nn as nn

In [2]:
input_image=torch.rand(1,3,224,224)
print(input_image.shape)

torch.Size([1, 3, 224, 224])


![](Inception.png)
![](module.png)

In [62]:
class Inception_block(nn.Module):
    def __init__(self,in_channels,filters):
        super(Inception_block,self).__init__()

        self.branch_1x1=nn.Conv2d(in_channels=in_channels,out_channels=filters[0],kernel_size=1)

        self.branch_3x3_reduce=nn.Conv2d(in_channels=in_channels,out_channels=filters[1],kernel_size=1)
        self.branch_3x3=nn.Conv2d(in_channels=filters[1],out_channels=filters[2],kernel_size=3,padding=1)

        self.branch_5x5_reduce=nn.Conv2d(in_channels=in_channels,out_channels=filters[3],kernel_size=1)
        self.branch_5x5=nn.Conv2d(in_channels=filters[3],out_channels=filters[4],kernel_size=5,padding=2)  

        self.branch_maxpool=nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
        self.branch_maxpool_reduce=nn.Conv2d(in_channels=in_channels,out_channels=filters[5],kernel_size=1)  

    def forward(self,x):

        b1=self.branch_1x1(x)
#         print(b1.shape)
        b2=self.branch_3x3_reduce(x)
        b2=self.branch_3x3(b2)
#         print(b2.shape)
        b3=self.branch_5x5_reduce(x)
        b3=self.branch_5x5(b3)
#         print(b3.shape)
        b4=self.branch_maxpool(x)
        b4=self.branch_maxpool_reduce(b4)
#         print(b4.shape)
        out=torch.cat([b1,b2,b3,b4],dim=1)

        return out

In [88]:
class GoogleNet(nn.Module):
    def __init__(self,num_class=1000):
        super(GoogleNet,self).__init__()
        
        self.conv7x7=nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3)
        self.pool1=nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.conv3x3_reduce=nn.Conv2d(in_channels=64,out_channels=64,kernel_size=1,padding=1)
        self.conv3x3=nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,padding=1)
        self.pool2=nn.MaxPool2d(kernel_size=3,stride=2)  
        
        self.inception_3a=Inception_block(192,[64,96,128,16,32,32])       
        self.inception_3b=Inception_block(256,[128,128,192,32,96,64])
        self.pool3=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)  
        
        self.inception_4a=Inception_block(480,[192,96,208,16,48,64])       
        self.inception_4b=Inception_block(512,[160,112,224,24,64,64])
        self.inception_4c=Inception_block(512,[128,128,256,24,64,64])       
        self.inception_4d=Inception_block(512,[112,144,288,32,64,64])
        self.inception_4e=Inception_block(528,[256,160,320,32,128,128])       
        self.pool4=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)   
        
        self.inception_5a=Inception_block(832,[256,160,320,32,128,128]) 
        self.inception_5b=Inception_block(832,[384,192,384,48,128,128])    
        self.pool5=nn.AvgPool2d(kernel_size=7,stride=1) 
        
        self.drop=nn.Dropout2d(p=0.4)
        
        self.fc=nn.Linear(1024,num_class)
        
    def forward(self,x):
        x=self.conv7x7(x)
        x=self.pool1(x)
        
        x=self.conv3x3_reduce(x)
        x=self.conv3x3(x)
        x=self.pool2(x)
        
        x=self.inception_3a(x)
        x=self.inception_3b(x)
        x=self.pool3(x)

        x=self.inception_4a(x)
        x=self.inception_4b(x)
        x=self.inception_4c(x)
        x=self.inception_4d(x)
        x=self.inception_4e(x)
        x=self.pool4(x)
        
        x=self.inception_5a(x)
        x=self.inception_5b(x)
        x=self.pool5(x)
        
        x=self.drop(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
#         print(x.shape)
        return x

In [89]:
net=GoogleNet(1000)
out=net(input_image)
print(out.shape)

torch.Size([1, 1000])


In [90]:
torch.save(net,'Googlenet.pth')

F:\anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GoogleNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
F:\anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Inception_block. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


![](Googlenet.png)